In [ ]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)

For both lambda = 0.01 and 0.08 do:
- Do Minimal basis pCQED (Nel = 2, Np = 2)
- Do large basis pCQED (Nel = 50, Np = 10)
- Do minimal basis QED-FCI (Np = 1)
- Do large basis QED-FCI (Np = 6)

In [ ]:
# options for mgf
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    "save_jk" : True
}


psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)


In [ ]:
mol_str = """
Li
H 1 1.5
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

pcqed_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 0
}

scqed_dict_np1 = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 1
}

scqed_dict_np6 = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 6
}


mol = psi4.geometry(mol_str)
psi4.set_options(options_dict)

pcqed_res = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    pcqed_dict
)

scqed_res_np1 = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    scqed_dict_np1
)

scqed_res_np6 = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    scqed_dict_np6
)

print(np.shape(pcqed_res.H_PF))
print(np.shape(scqed_res_np1.H_PF))
print(np.shape(scqed_res_np6.H_PF))

In [ ]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 40
N_el = 50
r_array = np.linspace(1.4, 2.0, N_R)

# for pcqed results
E_array = np.zeros((N_R, N_el))
Mu_array = np.zeros((N_R, N_el, N_el, 3))

# scqed results
sc_np1_E_array = np.zeros((N_R, 10))
sc_np6_E_array = np.zeros((N_R, 10))

psi4.set_options(options_dict)

r_idx = 0
for r in r_array:
    mol_str = mol_tmpl.replace("**R**", str(r))
    print(mol_str)
    mol = psi4.geometry(mol_str)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy = psi4.energy('fci',ref_wfn=wfn)
    
    # pcqed results
    pcqed_res = PFHamiltonianGenerator(mol_str, options_dict, pcqed_dict)
    assert np.isclose(fci_energy, pcqed_res.CIeigs[0], 1e-7)
    
    # sort out states with dipole-allowed transition from ground state
    singlet_states = pcqed_res.sort_dipole_allowed_states( N_el)
    
    # store values
    E_array[r_idx, :] = pcqed_res.CIeigs[singlet_states]
    Mu_array[r_idx, :, :, :] = pcqed_res.compute_dipole_moments(singlet_states)
    
    # scqed results
    cqed_res_np1 = PFHamiltonianGenerator(mol_str, options_dict, scqed_dict_np1)
    sc_np1_E_array[r_idx, :] = cqed_res_np1.CIeigs[:10]
    
    cqed_res_np6 = PFHamiltonianGenerator(mol_str, options_dict, scqed_dict_np6)
    sc_np6_E_array[r_idx, :] = cqed_res_np6.CIeigs[:10]
    

    
    r_idx += 1



In [ ]:
from matplotlib import pyplot as plt

safe_np1_1 = np.copy(sc_np1_E_array[:,1])
safe_np1_2 = np.copy(sc_np1_E_array[:,2])

safe_np6_1 = np.copy(sc_np6_E_array[:,1])
safe_np6_2 = np.copy(sc_np6_E_array[:,2])





In [ ]:
np.save("LiH_STO3G_FCI_E_Array", E_array)
np.save("LiH_STO3G_FCI_Mu_Array", Mu_array)

In [ ]:
new_E_array = np.load("LiH_STO3G_FCI_E_Array.npy")
new_Mu_array = np.load("LiH_STO3G_FCI_Mu_Array.npy")

In [ ]:
assert np.allclose(new_Mu_array, Mu_array)

In [ ]:
sc_np1_E_array[:,1] = np.copy(safe_np1_1)
sc_np1_E_array[:,2] = np.copy(safe_np1_2)

sc_np6_E_array[:,1] = np.copy(safe_np6_1)
sc_np6_E_array[:,2] = np.copy(safe_np6_2)


In [ ]:
ns = 14
np_1_lp = np.copy(sc_np1_E_array[:ns,1])
np_1_up = np.copy(sc_np1_E_array[:ns,2])

np_6_lp = np.copy(sc_np6_E_array[:ns,1])
np_6_up = np.copy(sc_np6_E_array[:ns,2])



In [ ]:
sc_np1_E_array[:ns,1] = np.copy(np_1_up)
sc_np1_E_array[:ns,2] = np.copy(np_1_lp)

sc_np6_E_array[:ns,1] = np.copy(np_6_up)
sc_np6_E_array[:ns,2] = np.copy(np_6_lp)

In [ ]:
plt.plot(r_array, E_array[:,0]+0.1208, label="E0")
plt.plot(r_array, E_array[:,1], label="E1")
plt.plot(r_array, E_array[:,2], label="E2")
plt.show()


#plt.plot(r_array, sc_np1_E_array[:,0]+0.1208, label="E0")
plt.plot(r_array, sc_np1_E_array[:,1], 'ro', label="E1")
plt.plot(r_array, sc_np1_E_array[:,2], 'bo', label="E2")
plt.show()

#plt.plot(r_array, sc_np6_E_array[:,0]+0.1208, label="E0")
plt.plot(r_array, sc_np6_E_array[:,1], 'ro', label="E1")
plt.plot(r_array, sc_np6_E_array[:,2], 'bo', label="E2")
plt.show()



\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
def build_pf_hamiltonian(n_el, n_ph, E_R, omega, lamvec, mu):
    """
    Given an array of n_el E_R values and an n_ph states with fundamental energy omega
    build the PF Hamiltonian
    
    n_el : int
        the number of electronic states (n_el = 1 means only ground-state)
    
    n_ph : int
        the number of photon occupation states (n_ph = 1 means only the |0> state)
    
    E_R : np.array of floats
        the electronic energies
        
    omega : float
        the energy of the photonic mode
        
    lamvec : np.array of floats
        the lambda vector
        
    mu : (n_el x n_el x 3) np.array of floats 
        mu[i, j, k] is the kth cartesian component of the dipole moment expectation value between 
        state i and state j
    
    """
    H_PF = np.zeros((n_el * n_ph, n_el * n_ph))
    
    # take care of the diagonals first
    # bare electronic and photonic energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            H_PF[na,na] = E_R[a] + n * omega
        
    # diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for g in range(n_el):
                H_PF[na,na] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g,a,:])
            
    # off-diagonal dipole self energy
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            for b in range(n_el):
                nb = n * n_el + b
                for g in range(n_el):
                    if a != b:
                        H_PF[na, nb] += 0.5 * np.dot(lamvec, mu[a,g,:]) * np.dot(lamvec, mu[g, b, :])
                
    # off-diagonal bilinear coupling
    for n in range(n_ph):
        for a in range(n_el):
            na = n * n_el + a
            
            for m in range(n_ph):
                for b in range(n_el):
                    mb = m * n_el + b
                    
                    if n == (m-1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na,mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m) 
                        
                    elif n == (m+1) and a != b:
                        #print(n, a, na, m, b, mb)
                        H_PF[na, mb] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1) 
                        H_PF[mb, na] = -np.sqrt(omega / 2) * np.dot(lamvec, mu[a,b,:]) * np.sqrt(m+1)
                        
    
    return H_PF


def MAE(E, Ep):
    """
    function to return the mean absolute error of the approximate PES stored in Ep relative to 
    the exact PES stored in E
    """
    # get length
    _NR = len(E)
    
    # take element-by-element difference
    _diff = E - Ep
    
    # compute absolute value of differences
    _absdiff = np.abs(_diff)
    
    # return sum divided by _NR
    return np.sum(_absdiff) / _NR
                
            

        
    
    

\begin{equation}
\hat{H}_{\alpha n, \beta m} = \left( E_{\alpha}(R) + n \omega \right) \delta_{\alpha \beta} \delta_{nm} + \sqrt{\frac{\omega}{2}} \lambda \cdot \mu_{\alpha \beta}(R) \left(\sqrt{n} \delta_{n,m-1} + \sqrt{n+1} \delta_{n,m+1} \right)  \\
+\frac{1}{2} \sum_{\gamma = 1}^N \lambda \cdot \mu_{\alpha, \gamma}(R) \lambda \cdot \mu_{\gamma, \beta} \delta_{nm}
\end{equation}


In [ ]:
np.set_printoptions(precision=6, linewidth=200, suppress=True)
#omega_cav = 0.12086
#lambda_vector = np.array([0., 0., 0.08])
#N_el = 3
#N_ph = 2
omega_cav = 0.12086
lambda_vector = np.array([0, 0.0, 0.05])
N_el = 2
N_ph = 2

_HPF = build_pf_hamiltonian(N_el, N_ph, E_array[0,:], omega_cav, lambda_vector, Mu_array[0, :, :, :])
#print("H")
#print(_HPF)

_HPF_EDSE = np.array([
[-7.857786, -0.004725,  0.,       -0.011859],
[-0.004725, -7.729184, -0.011859,  0.      ],
[ 0.,       -0.011859, -7.736926, -0.004725],
[-0.011859,  0.,       -0.004725, -7.608324]
])


In [ ]:
print(_HPF[0,0]-_HPF_EDSE[0,0])

In [ ]:
N_el = 50
N_ph = 20
pcqed_nel50_np20 = np.zeros((N_R, N_el * N_ph ))
for k in range(N_R):           
    _HPF = build_pf_hamiltonian(N_el, N_ph, E_array[k,:], omega_cav, lambda_vector, Mu_array[k, :, :, :])
    pf_e, pf_c = np.linalg.eigh(_HPF)
    pcqed_nel50_np20[k,:] = pf_e



In [ ]:
from matplotlib import cm
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 12

In [ ]:
#plt.plot(r_array, Eg_QED_FCI_Np_1)
#sc_E_arrat_Np_6 = np.copy(sc_E_array)
plt.plot(r_array, E_array[:,0], 'black', linestyle="dashed", label="No Cavity")
plt.plot(r_array, pcqed_nel2_np2[:,0], 'red', label="pCQED (2,2)")
plt.plot(r_array, pcqed_nel50_np7[:,0], 'blue', label="pCQED (50,7)")
plt.plot(r_array, pcqed_nel50_np20[:,0], 'purple', linestyle="dashed", label="pCQED (50,20)")
#plt.plot(r_array, pcqed_nel50_np7[:,0], color='black', linestyle='dashed', label="Eg pCQED (50,7)")
plt.plot(r_array, sc_np1_E_array[:,0], 'ro', ms = 3, mfc = 'none', label="scQED Np 2")
plt.plot(r_array, sc_np6_E_array[:,0], 'bo', ms = 6, mfc = 'none', label="scQED Np 7") # edgecolors='red')
plt.xlim(1.4, 2.0)
plt.ylim(-7.885, -7.855)
plt.ylabel("Energy (Hartree)")
plt.xlabel("Bondlength (Angstroms)")
plt.legend()
plt.savefig("sto3g_lih_ground_state_lambda_05.png")
plt.show()


In [ ]:
rcParams['font.size'] = 10
plt.plot(r_array, E_array[:,0]+0.12086, 'grey', linestyle="dashed", label="Uncoupled")
plt.plot(r_array, E_array[:,1], 'grey', linestyle="dashed")#, label="Ee Uncoupled")
plt.plot(r_array, pcqed_nel2_np2[:,1], 'red', label="pCQED (2,2)")
plt.plot(r_array, pcqed_nel2_np2[:,2], 'red')# label="pCQED (2,2)")
plt.plot(r_array, pcqed_nel50_np7[:,1], 'blue',  label="pCQED (50,7)")
plt.plot(r_array, pcqed_nel50_np7[:,2], 'blue')#,  label="pCQED (50,7)")
plt.plot(r_array, sc_np1_E_array[:,2], 'ro', ms = 3, mfc = 'none', label="scQED Np 1")
plt.plot(r_array, sc_np1_E_array[:,3], 'ro', ms = 3, mfc = 'none')#, label="scQED Np 1")
plt.plot(r_array, sc_np6_E_array[:,2], 'bo', ms = 6, mfc = 'none', label="scQED Np 7")
plt.plot(r_array, sc_np6_E_array[:,3], 'bo', ms = 6, mfc = 'none')#, label="scQED Np 7")
plt.xlim(1.4, 2.0)
plt.ylim(-7.765, -7.725)
plt.ylabel("Energy (Hartree)")
plt.xlabel("Bondlength (Angstroms)")
plt.legend()
plt.savefig("sto3g_lih_lp_up_lambda_05.png")
plt.show()

In [ ]:
plt.plot(r_array, pcqed_nel50_np6[:,1], 'r--', label="LP pCQED (50,6)")
plt.plot(r_array, pcqed_nel50_np6[:,2], 'b--', label="UP pCQED (50,6)")
plt.plot(r_array, sc_E_arrat_Np_6[:,2], 'ro', label="LP CQED-FCI Np 6")
plt.plot(r_array, sc_E_arrat_Np_6[:,3], 'bo', label="UP CQED-FCI Np 6")
plt.ylabel("Energy (Hartree)")
plt.xlabel("Bondlength (Angstroms)")
plt.legend()
plt.show()

In [ ]:
for i in range(N_R):
    print(F'{r_array[i]}, {pf_array[i,0]}, {pf_array[i,1]}, {pf_array[i,2]}, {pf_array[i,3]}, {pf_array[i,3]} ')
      